In [4]:
import pandas as pd

from database_connect import connect_db

### Afficher le contenu du fichier movies

In [5]:
data_movie = pd.read_json('movie_ids_10_16_2024.json', lines=True)
data_movie

,adult,id,original_title,popularity,video
0,False,3924,Blondie,2.898,False
1,False,6124,Der Mann ohne Namen,3.167,False
2,False,8773,L'Amour à vingt ans,16.787,False
3,False,25449,New World Disorder 9: Never Enough,5.274,False
4,False,31975,Sesame Street: Elmo Loves You!,0.004,True
...,...,...,...,...,...
980183,False,1371993,오늘부터 1일,0.000,False
980184,False,1371994,Isolation,0.000,False
980185,False,1371995,The American Miracle,0.000,False
980186,False,1371996,Josh Barnett's Bloodsport XII,0.000,False


### Injection des données dans la base de données

In [6]:
connection = connect_db()

if connection:
    cursor = connection.cursor()

    # Insert data into the database
    try:
        # Begin a transaction
        cursor.fast_executemany = True
        insert_query = "INSERT INTO movies (id, adult, original_title, popularity, video) VALUES (?, ?, ?, ?, ?)"
        data_to_insert = [(row['id'], row['adult'], row['original_title'], row['popularity'], row['video']) for
                          index, row in data_movie.iterrows()]

        cursor.executemany(insert_query, data_to_insert)
        connection.commit()
        print("Données insérées avec succès dans la base de données.")
    except Exception as e:
        print(f"Erreur lors de l'insertion des données : {e}")
    finally:
        connection.close()
else:
    print("Connexion à la base de données échouée.")

Connexion réussie à la base de données SQL Server.
Données insérées avec succès dans la base de données.


# MISE A JOUR DE LA BASE DE DONNEES SCRIPT

### Telechargement du fichier

In [12]:
import shutil
import gzip
import requests
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d_%m_%Y")

# Variables
# url = f"http://files.tmdb.org/p/exports/movie_ids_{formatted_date}.json.gz"
url = "http://files.tmdb.org/p/exports/movie_ids_10_16_2024.json.gz"
# gz_file = f"movie_ids_{formatted_date}.json.gz"
gz_file = "movie_ids_10_16_2024.json.gz"
# json_file = f"movie_ids_{formatted_date}.json"
json_file = "movie_ids_10_16_2024.json"

# Télécharger le fichier
response = requests.get(url, stream=True)
with open(gz_file, "wb") as f:
    f.write(response.content)

# Extraire le fichier gz
with gzip.open(gz_file, "rb") as f_in:
    with open(json_file, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

### Dataframe and update

In [20]:
from update_script import update_movies_table

data_movie_update = pd.read_json(json_file, lines=True)
update_movies_table(data_movie_update)
data_movie_update

TypeError: update_movies_table() missing 1 required positional argument: 'data_movie'

### Insertion en base de données + MAJ ( tous les 3 mois )

TypeError: update_movies_table() missing 1 required positional argument: 'data_movie'